In [ ]:
#import libraries
import imdlib as imd
import geopandas as gpd
import regionmask
import numpy as np
import pandas as pd
import rioxarray as rio

In [ ]:
#Download IMD grided rainfall data
start_dy = '2019-01'
end_dy = '2020-07'
var_type = 'rain'
file_dir='C:/Users/DELL/Desktop/RMSI_test/rain'
data = imd.get_real_data(var_type, start_dy, end_dy, file_dir)


In [ ]:
#read and check the data
data = imd.open_real_data(var_type, start_dy, end_dy, file_dir)

ds = data.get_xarray()
print(ds)
#data.to_geotiff('test.tif', file_dir)

In [ ]:
#plotting the data
ds = ds.where(ds['rain'] != -999.) #Remove NaN values
#dss=ds['rain'].max('time').plot()
for i in range(30):

    ds2d = ds['rain'].isel(time=i)
    ds2d.rio.to_raster(str(ds['time'][i])[36:46] + "_rain.tif")

    


In [ ]:
for i in range(30):
    date= ds['time'][i]
    print(date)

In [ ]:
str(ds['time'][i])[36:46]


In [ ]:
pip install rioxarray

In [ ]:
#import the India state boundary shapefile
state_boun = gpd.read_file("C:/Users/DELL/Desktop/RMSI_test/india_state/indstate/stateboundary.zip")
state_boun["State_Name"]


In [ ]:
# mask the rainfall data statewise
state_mask = regionmask.mask_3D_geopandas(state_boun,
                                         ds.lon,
                                         ds.lat)
state_mask

rain_state_boun_mask = ds["rain"].where(state_mask)

rain_state_boun_mask

In [ ]:
#extract the maximum rain fall
dff=rain_state_boun_mask.groupby("region").max(["time","lat", "lon"]).to_dataframe()

In [ ]:
#Add the maximum rainfall to the state boundary shapefile
gdf = state_boun.assign(Max_Rain=dff['rain'])
gdf


In [ ]:
#plot the maximum daily rainfall state wise

map1=gdf.plot(column='Max_Rain', legend=True, legend_kwds={'label': "Daily maximum rainfall (mm)",
                        'orientation': "vertical"}).get_figure().savefig('C:/Users/DELL/Desktop/RMSI_test/Ind_stat_rain.png', bbox_inches='tight', dpi=300)

gdf.plot.bar(x='State_Name', y='Max_Rain', rot=90, figsize= [12,4], ylabel= 'Rainfall (mm/day)', xlabel='State Name', title='State wise monthly daily maximum rainfall ').get_figure().savefig('C:/Users/DELL/Desktop/RMSI_test/stat_rain.png', bbox_inches='tight', dpi=300)